In [3]:
import pyspark
import pandas as pd
import numpy as np
import pyspark.sql.functions as F

spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/20 14:15:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/20 14:15:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Create a spark data frame that contains your favorite programming languages.

    The name of the column should be language
    View the schema of the dataframe
    Output the shape of the dataframe
    Show the first 5 records in the dataframe

In [5]:
from pyspark.sql.functions import col, expr


In [12]:
language_list = ['python', 'java', 'C+', 'unix', 'linux', 'something_else']

In [19]:
lang_df = pd.DataFrame(language_list, columns=['language'])

In [20]:
df = spark.createDataFrame(lang_df)

In [21]:
df.printSchema()


root
 |-- language: string (nullable = true)



In [23]:
print(df.count(), "rows", len(df.columns), "columns")


6 rows 1 columns


In [24]:
df.show(5)

+--------+
|language|
+--------+
|  python|
|    java|
|      C+|
|    unix|
|   linux|
+--------+
only showing top 5 rows



# Load the mpg dataset as a spark dataframe.

In [25]:
from pydataset import data
mpg = spark.createDataFrame(data('mpg'))

## Create 1 column of output that contains a message like the one below:
    The 1999 audi a4 has a 4 cylinder engine.
## For each vehicle

In [26]:
mpg.show(1)

+------------+-----+-----+----+---+--------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|   trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+--------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|auto(l5)|  f| 18| 29|  p|compact|
+------------+-----+-----+----+---+--------+---+---+---+---+-------+
only showing top 1 row



In [32]:
mpg.createOrReplaceTempView("mpg")

In [41]:
spark.sql("""
SELECT CONCAT(
        'The ',
        year,
        ' ',
        manufacturer,
        ' ',
        model,
        ' has a ',
        cyl,
        ' cylinder engine.') as Statement
FROM mpg
"""
).show(5, truncate=False)

+-----------------------------------------+
|Statement                                |
+-----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 6 cylinder engine.|
+-----------------------------------------+
only showing top 5 rows



# Transform the trans column so that it only contains either manual or auto.

In [42]:
from pyspark.sql.functions import when

In [50]:
mpg.select(mpg.trans, when(mpg.trans.like('%auto%'), "auto")
          .otherwise('manual').alias('trans')).show(12)

+----------+------+
|     trans| trans|
+----------+------+
|  auto(l5)|  auto|
|manual(m5)|manual|
|manual(m6)|manual|
|  auto(av)|  auto|
|  auto(l5)|  auto|
|manual(m5)|manual|
|  auto(av)|  auto|
|manual(m5)|manual|
|  auto(l5)|  auto|
|manual(m6)|manual|
|  auto(s6)|  auto|
|  auto(l5)|  auto|
+----------+------+
only showing top 12 rows



# Load the tips dataset as a spark dataframe.

In [51]:
tips = spark.createDataFrame(data('tips'))

In [52]:
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



# What percentage of observations are smokers?


In [53]:
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean


In [61]:
tips.select((count(tips.smoker == 'Yes') / (len(tips.columns))).alias('% of smokers')).show()

+------------------+
|      % of smokers|
+------------------+
|34.857142857142854|
+------------------+



# Create a column that contains the tip percentage

In [69]:
tips.select(tips.total_bill, tips.tip, (tips.tip / tips.total_bill).alias('tip_percentage')).show()

+----------+----+-------------------+
|total_bill| tip|     tip_percentage|
+----------+----+-------------------+
|     16.99|1.01|0.05944673337257211|
|     10.34|1.66|0.16054158607350097|
|     21.01| 3.5|0.16658733936220846|
|     23.68|3.31| 0.1397804054054054|
|     24.59|3.61|0.14680764538430255|
|     25.29|4.71|0.18623962040332148|
|      8.77| 2.0|0.22805017103762829|
|     26.88|3.12|0.11607142857142858|
|     15.04|1.96|0.13031914893617022|
|     14.78|3.23| 0.2185385656292287|
|     10.27|1.71| 0.1665043816942551|
|     35.26| 5.0|0.14180374361883155|
|     15.42|1.57|0.10181582360570687|
|     18.43| 3.0|0.16277807921866522|
|     14.83|3.02|0.20364126770060686|
|     21.58|3.92|0.18164967562557924|
|     10.33|1.67| 0.1616650532429816|
|     16.29|3.71|0.22774708410067526|
|     16.97| 3.5|0.20624631703005306|
|     20.65|3.35|0.16222760290556903|
+----------+----+-------------------+
only showing top 20 rows



# Calculate the average tip percentage for each combination of sex and smoker.


In [75]:
tips.rollup('sex', 'smoker').agg(mean((tips.tip / tips.total_bill)).alias('average_tip_percentage')).show()

+------+------+----------------------+
|   sex|smoker|average_tip_percentage|
+------+------+----------------------+
|Female|  null|   0.16649073632892478|
|  Male|    No|    0.1606687151291298|
|  null|  null|   0.16080258172250475|
|Female|    No|   0.15692097076918363|
|  Male|  null|   0.15765054700429745|
|  Male|   Yes|    0.1527711752024851|
|Female|   Yes|   0.18215035269941032|
+------+------+----------------------+



# Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [80]:
from vega_datasets import data
weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)


In [81]:
weather.show(1)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
+----------+-------------+--------+--------+----+-------+
only showing top 1 row



# Convert the temperatures to fahrenheit.

In [82]:
weather.select((weather.temp_max * 9/5 + 32).alias('temp_max_f'), 
               (weather.temp_min * 9/5 + 32).alias('temp_min_f')).show(5)

+----------+----------+
|temp_max_f|temp_min_f|
+----------+----------+
|     55.04|      41.0|
|     51.08|     37.04|
|     53.06|     44.96|
|     53.96|     42.08|
|     48.02|     37.04|
+----------+----------+
only showing top 5 rows



# Which month has the most rain, on average?

In [87]:
from pyspark.sql.functions import month, year, quarter
from pyspark.sql.functions import asc, desc


In [96]:
(
    weather.withColumn("month", month("date"))
    .groupBy("month")
    .agg(mean("precipitation").alias("mean_rainfall"))
    .sort(desc("mean_rainfall"))
    .show()

)

+-----+-------------------+
|month|      mean_rainfall|
+-----+-------------------+
|   11|  5.354166666666667|
|   12|  5.021774193548388|
|    3|  4.888709677419355|
|   10|  4.059677419354839|
|    1| 3.7580645161290316|
|    2|  3.734513274336283|
|    4|  3.128333333333333|
|    9| 1.9624999999999997|
|    5| 1.6733870967741935|
|    8| 1.3201612903225806|
|    6| 1.1075000000000002|
|    7|0.38870967741935486|
+-----+-------------------+



# Which year was the windiest?

In [98]:
(
    weather.withColumn("year", year("date"))
    .groupBy("year")
    .agg(mean("wind").alias("annual_wind"))
    .sort(desc("annual_wind"))
    .show()

)

+----+------------------+
|year|       annual_wind|
+----+------------------+
|2012| 3.400819672131148|
|2014|3.3876712328767122|
|2015|3.1597260273972596|
|2013|3.0158904109589044|
+----+------------------+



# What is the most frequent type of weather in January?


In [107]:
(
    weather.filter(month("date") == 1)
    .groupBy("weather")
    .agg(count("weather"))
    .show()
)

+-------+--------------+
|weather|count(weather)|
+-------+--------------+
|drizzle|            10|
|   rain|            35|
|    sun|            33|
|   snow|             8|
|    fog|            38|
+-------+--------------+



# What is the average high and low temperature on sunny days in July in 2013 and 2014?

In [122]:
(
    weather.filter(month("date") == 7)
    .filter((year('date') == 2013) | (year('date') == 2014))
    .filter(weather.weather == 'sun')
    .groupBy(year('date'))
    .agg(mean("temp_max"), mean('temp_min'))
    .show()
)

+----------+------------------+------------------+
|year(date)|     avg(temp_max)|     avg(temp_min)|
+----------+------------------+------------------+
|      2013|26.585185185185193|13.981481481481483|
|      2014|            27.092|14.400000000000002|
+----------+------------------+------------------+



# What percentage of days were rainy in q3 of 2015?


In [169]:


(
    weather.filter(year("date") == 2015)
    .filter(quarter('date') == 3)
    .withColumn("rainy", (weather.weather == 'rain').cast("int"))
    .groupBy(quarter('date'))
    .agg(mean('rainy') * 100)
    
    .show()
)

+-------------+------------------+
|quarter(date)|(avg(rainy) * 100)|
+-------------+------------------+
|            3|2.1739130434782608|
+-------------+------------------+



# For each year, find what percentage of days it rained (had non-zero precipitation).


In [172]:
(
    weather.withColumn("rainy", (weather.weather != 'sun').cast("int"))
    .agg(mean('rainy') * 100)
    
    .show()
)

+------------------+
|(avg(rainy) * 100)|
+------------------+
|51.129363449691986|
+------------------+



In [171]:
weather.show(100)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|
|2012-01-10|          1.0|     6.1|     0.6| 3.4|   rain|
|2012-01-11|          0.0|     6.1|    -1.1| 5.1|    sun|
|2012-01-12|          0.0|     6.1|    -1.7| 1.9|    sun|
|2012-01-13|          0.0|     5.0|    -2.8| 1.3|    sun|
|2012-01-14|          4.1|     4.4|     0.6| 5.3|   snow|
|2012-01-15|  